In [1]:
def create_visualization(data_df,user_id,week_num,data_summary_user_current):
    import pandas as pd
    import plotly.express as px
    import pickle, random
    import copy


    
    data_df=data_df.loc[data_df['Stressor'].notnull()]
    user_df=data_df.loc[data_df.user==user_id]
    with open('/home/jupyter/sneupane/MOODS/pickled_files/colors.pickle', 'rb') as handle:
        colors = pickle.load(handle)
    random.shuffle(colors)
    colors_stressors=['#00429d', '#73a2c6', '#ffffe0', '#b11744', '#93003a']
    random.shuffle(colors_stressors)
    colors_time_of_day=['#4f1f2b', '#b57b86', '#b476ff', '#0000d8']
    random.shuffle(colors_time_of_day)
#     print(len(user_df))
    if len(user_df )> 0:
        temp_df=user_df.groupby(["Stressor"],as_index=False)["counter"].count()
        
        temp_df=temp_df.sort_values(["counter"],ascending=False)
        temp_df=temp_df.head(5)
        top_stressors=temp_df["Stressor"].values.tolist()
        
        user_df=user_df.loc[user_df["Stressor"].isin(top_stressors)]
        print(len(user_df))
        
        top_stressors=[strr.replace(' ', '<br>') for strr in top_stressors]
        
        modified_stressors=[]
        for strr in user_df.Stressor.values.tolist():
            modified_stressors.append(strr.replace(' ', '<br>') )
        user_df["Stressor"]=modified_stressors
        user_df.Location = user_df.Location.fillna('Unknown')
        
        user_df=user_df.groupby(["Stressor",'Location','start_time_of_day']).count()
        
#         user_df["color"]=
        user_df=user_df.reset_index()
        color_map={}
#         count=1
        final_list=[]
        
        locations_list=user_df["Location"].unique().tolist()
        time_list=user_df["start_time_of_day"].unique().tolist()
        final_list.extend(top_stressors)
        final_list.extend(locations_list)
        final_list.extend(time_list)
        count,count1,count2=0,0,0
        
        
        
#         top_stressors_1=top_stressors.copy()
#         locations_list_1=locations_list.copy()
        
        for item in final_list:
#             print(item)
            if item in top_stressors :
               
                if item not in color_map:
                    color_map[item]=colors_stressors[count]
                    count+=1
                
            elif item in time_list:
               
                color_map[item]=colors_time_of_day[count1]
                count1+=1
            elif item in locations_list:
                if item not in color_map:
                    color_map[item]=colors[count2]
                    count2+=1
                
#         modified_stressors=[]
#         print(user_df.Stressor.values.tolist())
        
        fig=px.sunburst(user_df, path=[ 'Stressor', 'Location','start_time_of_day'], values='counter',color='Stressor',
                         color_discrete_map=color_map,
                        hover_name="Stressor",hover_data=["Location", "start_time_of_day"])
        
        labels=fig["data"][0]["labels"]
        custom_colors=[]
        font_test=[]
        labels_modified=[]
#         print(top_stressors)
        for label in labels:
            custom_colors.append(color_map[label])
            if label in top_stressors:
#                 font_test.append(font_dict["top_stressors"])
                labels_modified.append("<b><i>"+label+"</i></b>")
            elif label in locations_list:
#                 font_test.append(font_dict["Location"])
                labels_modified.append("<b>"+label+"</b>")
            elif label in time_list:
#                 font_test.append(font_dict["start_time_of_day"]) 
                labels_modified.append("<i>"+label+"</i>")
        print("I")
        fig.update_layout(hovermode="closest",
                    autosize=True,

                    margin=dict(
                        l=0,
                        r=0,
                        b=0,
                        t=0,
#                         pad=20
                    ),)
        fig.update_traces(
   # hovertemplate=None,
#    hoverinfo='skip'
)
        
        fig["data"][0]["labels"]=labels_modified
        fig.update_traces(hovertemplate=
                                      "<b>Stressor:</b> %{customdata[2]} <br>" +
                                "<b>Location:</b> %{customdata[0]}<br>" +
                                "<b>Time of day: </b>%{customdata[1]}<br>" +        
                                "<extra></extra>", selector=dict(type='sunburst'))
        fig.update_layout(uniformtext=dict(minsize=6, mode='hide'))
        fig.update_traces(leaf_opacity=1, selector=dict(type='sunburst'))
        fig.update_traces(marker_line_color="black", selector=dict(type='sunburst'),marker_line_width=2,marker_colors=custom_colors)
#         fig.show()
#         print(fig)
        
        temp_df_1=user_df.loc[user_df["week"]!=week_num]
        temp_df_1=temp_df_1.groupby(["Stressor"],as_index=False)["counter"].count()
        temp_df_1=temp_df_1.sort_values(["counter"],ascending=False)
        temp_df_1=temp_df_1.head(5)
        top_stressors_lastweeks=temp_df["Stressor"].values.tolist()
        difference=list(set(top_stressors) - set(top_stressors_lastweeks))
        data_summary_user=data_summary_user_current["Prominent Stressor Context"]
        if len(difference) > 0:
            data_summary_user=data_summary_user.replace("_b_", str(" ,".join(difference)))
        else:
            data_summary_user=data_summary_user.replace("_b_", "No change in the most frequent list this week")
        
        data_summary_user=data_summary_user.replace("_a_", str(" ,".join(top_stressors)))
        data_summary_user=data_summary_user.replace("week_num", str(week_num))
        data_summary_user=""
        return "Prominent Stressor Context",fig,data_summary_user
    
    else:
        print("Hello")
        return "","",""
     